In [29]:
import cv2
import numpy as np

In [55]:
bg = cv2.imread('blendBG.png')
fg = cv2.imread('blendFG.png')
mask = cv2.imread('mask1.png',0)
blah = cv2.imread('mask1.png')

print(np.shape(mask))

width, height, _ = np.shape(bg)

# small = cv2.resize(fg, (0,0), fx=0.25, fy=0.25)
# smallmask = cv2.resize(mask, (0,0), fx=0.25, fy=0.25)

im2, contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

x,y,w,h = cv2.boundingRect(contours[0])

x1 = x + w
y1 = y + h

# if more then 1 contour is found
for i in range(1, len(contours)):

    # find overall bounding rectangle
    x_, y_, w_, h_ = cv2.boundingRect(contours[i])
    
    # convert from (x,w) to (x,y) system
    # new end point
    x2 = x_ + w_
    y2 = y_ + h_
    
    #top line
    if x_ < x:
        x = x_
    #left line
    if y_ < y:
        y = y_
    #bottom line
    if x2 > x1:
        x1 = x2 
    #right line
    if y2 > y1:
        y1 = y2
    
# print(np.shape(contours))
# sumCon = np.concatenate((contours[0], contours[1]))


# print(type(contours[0]))
# out = cv2.drawContours(blah, sumCon, -1, (0,255,0), 7)

# cv2.imshow("rect", blah)
# k = cv2.waitKey(0) & 0xff

# cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

w = x1 - x
h = y1 - y

result = cv2.seamlessClone(fg, bg, mask, (int(x+w/2), int(y+h/2)), cv2.NORMAL_CLONE)

# print(np.shape(result))

cv2.imshow("result", result)
k = cv2.waitKey(0) & 0xff

cv2.destroyAllWindows()

(1080, 1920)


In [4]:
def blend_image(bg, fg, mask):
    """
    Poisson blending of the foreground object into the background image 
    using seamless clone function of OpenCV

    Args:
        bg: Colour background image 
        fg: Colour foreground object with same dimensions aas bg
        mask: Greyscale mask of the foreground object

    Returns: The blended image
    """
    
    # We need to find the center of a rectangle that bounds all the white pixels in the mask.
    # The center is passed as a parameter to the seamless blending function to 
    # position the object in the same position as in the mask.
    
    width, height, _ = np.shape(bg)
    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # find the bounding points of the contour
    x,y,w,h = cv2.boundingRect(contours[0])
    
    # bottom right co-ordinates
    x1 = x + w
    y1 = y + h

    # if more then 1 contour is found, draw a bounding rectangle 
    # around the entire set of countours
    for i in range(1, len(contours)):

        # find bounding rectangle
        x_, y_, w_, h_ = cv2.boundingRect(contours[i])

        # convert from (x,w) to (x,y) system
        # new end point
        x2 = x_ + w_
        y2 = y_ + h_

        # update top edge
        if x_ < x:
            x = x_
        # update left edge
        if y_ < y:
            y = y_
        # update bottom edge
        if x2 > x1:
            x1 = x2 
        # update right line
        if y2 > y1:
            y1 = y2
    
    # Find the updated width and height using the new bottom-right point
    w = x1 - x
    h = y1 - y

    result = cv2.seamlessClone(fg, bg, mask, (int(x+w/2), int(y+h/2)), cv2.NORMAL_CLONE)
    
    return result

In [ ]:
cv2.cop